<a href="https://colab.research.google.com/github/UScodes/FineTuneILama3.1-8b/blob/main/Ilama3_1trainedwithLora.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install -q transformers datasets peft bitsandbytes accelerate


In [2]:
import os
import json
import torch
from datasets import Dataset
from transformers import AutoTokenizer, AutoModelForCausalLM, TrainingArguments, Trainer, DataCollatorForLanguageModeling
from peft import get_peft_model, LoraConfig, TaskType
from huggingface_hub import notebook_login

In [ ]:
# 1. Clear model from memory (if it exists)
if 'model' in locals():
    del model
if torch.cuda.is_available():
    torch.cuda.empty_cache()
print("✅ Memory cleared.")

✅ Memory cleared.


In [3]:
# 1. Set environment variable for memory fragmentation
os.environ["PYTORCH_CUDA_ALLOC_CONF"] = "expandable_segments:True"

In [4]:
# 2. Hugging Face login
notebook_login()

In [5]:
# 3. Model identifier
model_id = "meta-llama/Llama-3.1-8B"

In [6]:
# 4. Load tokenizer
try:
    tokenizer = AutoTokenizer.from_pretrained(model_id, use_auth_token=True)
    tokenizer.pad_token = tokenizer.eos_token
    print("✅ Tokenizer loaded.")
except Exception as e:
    print(f"❌ Error loading tokenizer: {e}")
    exit()

/usr/local/lib/python3.11/dist-packages/transformers/models/auto/tokenization_auto.py:898: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers. Please use `token` instead.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


✅ Tokenizer loaded.


In [7]:
# 5. Load your dataset (replace with your actual loading)
# Assuming your data is a list of dictionaries with 'instruction', 'input', 'output' keys
data = []
for file in os.listdir("/content"):
    if file.endswith(".json"):
        try:
            with open(f"/content/{file}", "r", encoding="utf-8") as f:
                content = json.load(f)
                if isinstance(content, list):
                    data.extend(content)
                else:
                    print(f"⚠️ Skipped {file} - not a list")
                print(f"✅ Loaded: {file} with {len(content)} items")
        except Exception as e:
            print(f"❌ Error loading {file}: {e}")

dataset = Dataset.from_list(data)

✅ Loaded: UnicodeUtilities.json with 55 items
✅ Loaded: OpModeHandler.json with 32 items
✅ Loaded: AbstractObject.json with 8 items
✅ Loaded: MessageInterfaces.json with 20 items
✅ Loaded: AssertionInterfaceLib.json with 107 items
✅ Loaded: TestOpModeHandler.json with 6 items
✅ Loaded: CollectionsTestLib.json with 19 items
✅ Loaded: CycleManager.json with 3 items
✅ Loaded: CycleManagerTest.json with 4 items
✅ Loaded: CollectionInterfaces.json with 57 items
✅ Loaded: OpModeInterfaces.json with 15 items
✅ Loaded: ConcreteMessage.json with 8 items
✅ Loaded: AbstractObjectTest.json with 2 items
✅ Loaded: CycleManagerInterfaces.json with 24 items
✅ Loaded: AssertionInterfaceTests.json with 45 items
✅ Loaded: TestUnicode.json with 3 items
✅ Loaded: Collection.json with 32 items
✅ Loaded: ReturnTypes.json with 6 items
✅ Loaded: AbstractObjectTestLib.json with 6 items


In [8]:
# 6. Tokenization with a potentially smaller max_length
def tokenize(example):
    prompt = f"User: {example['instruction']}\nInput: {example.get('input', '')}\nAssistant: {example['output']}"
    tokens = tokenizer(prompt, truncation=True, padding="max_length", max_length=256) # Reduced max_length
    tokens["labels"] = tokens["input_ids"].copy()
    return tokens

    tokenized_dataset = dataset.map(tokenize, remove_columns=dataset.column_names)

In [9]:
#  Apply tokenize safely with logging
try:
    tokenized_dataset = dataset.map(tokenize, remove_columns=dataset.column_names)
    print("✅ Tokenization completed.")
    if tokenized_dataset:
        print("🔍 Sample tokenized example:", tokenized_dataset[0])
    else:
        print("⚠️ tokenized_dataset is empty after mapping.")
except Exception as e:
    print(f"❌ Error during tokenization: {e}")
    exit()

Map:   0%|          | 0/452 [00:00<?, ? examples/s]

✅ Tokenization completed.
🔍 Sample tokenized example: {'input_ids': [128000, 1502, 25, 4324, 264, 734, 2663, 2175, 8780, 45521, 430, 4780, 279, 3160, 320, 258, 5943, 369, 44532, 12, 23, 11, 304, 4339, 369, 44532, 12, 845, 8, 323, 279, 3752, 1797, 315, 459, 5884, 707, 320, 30096, 854, 81125, 340, 2566, 25, 36047, 29296, 220, 18, 11, 1893, 2082, 198, 72803, 25, 674, 2175, 8780, 45521, 271, 8586, 374, 264, 734, 430, 4780, 3160, 304, 3878, 315, 5943, 369, 20677, 12, 23, 323, 304, 4339, 304, 3878, 315, 20677, 12, 845, 439, 1664, 439, 3752, 1797, 315, 925, 828, 955, 11, 5922, 311, 5296, 1618, 430, 433, 1587, 539, 1797, 854, 81125, 315, 279, 925, 382, 567, 55375, 52243, 1342, 198, 4163, 198, 8846, 12399, 198, 1547, 15092, 2028, 5865, 4780, 279, 3160, 320, 258, 5943, 369, 44532, 12, 23, 11, 304, 4339, 369, 44532, 12, 845, 8, 323, 279, 720, 19740, 1797, 315, 459, 5884, 707, 320, 30096, 854, 81125, 3677, 497, 6666, 487, 720, 197, 2460, 9246, 527, 18073, 439, 854, 12, 69759, 5027, 23914, 271, 497

In [10]:
# 7. Load the base model with bfloat16 and device map auto
try:
    model = AutoModelForCausalLM.from_pretrained(
        model_id,
        torch_dtype=torch.bfloat16 if torch.cuda.is_bf16_supported() else torch.float16 if torch.cuda.is_available() else torch.float32,
        device_map="auto",
        token=True
    )
    model.enable_input_require_grads() # Enable for gradient checkpointing
    print("✅ Model loaded.")
except Exception as e:
    print(f"❌ Error loading model: {e}")
    exit()

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

✅ Model loaded.


In [11]:
# 8. Apply minimal LoRA configuration
peft_config = LoraConfig(
    task_type=TaskType.CAUSAL_LM,
    inference_mode=False,
    r=4,             # Minimal LoRA rank
    lora_alpha=8,    # Minimal LoRA alpha
    lora_dropout=0.1,
  #   lora_all_linear=True, # Apply LoRA to all linear layers
)

model = get_peft_model(model, peft_config)
model.print_trainable_parameters()


trainable params: 1,703,936 || all params: 8,031,965,184 || trainable%: 0.0212


In [12]:
# 9. Define minimal memory training arguments
training_args = TrainingArguments(
    output_dir="./llama-ft-minimal",
    per_device_train_batch_size=1,     # Minimal batch size
    gradient_accumulation_steps=16,   # High accumulation
    logging_steps=10,
    save_steps=500,
    num_train_epochs=3,                # Adjust as needed
    report_to="none",
    fp16=False,                       # Let bf16 handle precision
    bf16=torch.cuda.is_bf16_supported(),
    gradient_checkpointing=True,    # Crucial for memory saving
    optim="adamw_torch",
    learning_rate=1e-4,              # Adjust as needed
    weight_decay=0.01,
    max_grad_norm=1.0,
    dataloader_num_workers=2,
    remove_unused_columns=False,
)

In [13]:
# 10. Data collator
data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=False)

In [14]:
# 11. Setup Trainer
try:
    trainer = Trainer(
        model=model,
        args=training_args,
        train_dataset=tokenized_dataset,
        data_collator=data_collator,
    )
    print("✅ Trainer set up.")
except Exception as e:
    print(f"❌ Error setting up Trainer: {e}")
    exit()

No label_names provided for model class `PeftModelForCausalLM`. Since `PeftModel` hides base models input arguments, if label_names is not given, label_names can't be set automatically within `Trainer`. Note that empty label_names list will be used instead.


✅ Trainer set up.


In [15]:
# 12. Train the model
try:
    trainer.train()
    print("✅ Training complete.")
except Exception as e:
    print(f"❌ Training failed: {e}")

`use_cache=True` is incompatible with gradient checkpointing. Setting `use_cache=False`.


Step,Training Loss
10,2.759000
20,2.569600
30,2.421300
40,2.121300
50,2.018500
60,1.826700
70,1.730800
80,1.705600


✅ Training complete.


In [ ]:
from peft import PeftModel
from transformers import AutoModelForCausalLM, AutoTokenizer
import torch

In [ ]:
# 1. Define the Colab output directory (adjust if you changed it)
output_dir = "./llama-ft-minimal/checkpoint-500" # Assuming you trained for at least 500 steps

In [ ]:
# 2. Load the base model and tokenizer
model_id = "meta-llama/Llama-3.1-8B"
tokenizer = AutoTokenizer.from_pretrained(model_id, use_auth_token=True)

/usr/local/lib/python3.11/dist-packages/transformers/models/auto/tokenization_auto.py:898: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers. Please use `token` instead.
  warnings.warn(


In [ ]:
# 3. Load the fine-tuned LoRA model
try:
    model = AutoModelForCausalLM.from_pretrained(model_id, torch_dtype=torch.bfloat16 if torch.cuda.is_bf16_supported() else torch.float16, device_map="auto", token=True)
    trained_model = PeftModel.from_pretrained(model, output_dir)
    print(f"✅ Loaded fine-tuned model from {output_dir}")
except Exception as e:
    print(f"❌ Error loading fine-tuned model: {e}")
    exit()

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

❌ Error loading fine-tuned model: Can't find 'adapter_config.json' at './llama-ft-minimal/checkpoint-500'
